# Gutenberg Exploration

This notebook is dedicated at cleaning and exploring the Gutenber project dataset which seems to be kind of difficult to deal with.

There are a few things that I want to explore and save about each book:

Gutenberg Index and file used for the analysis
Character set encoding: UTF-8

Title
Author
Language

Total book Length (characters, words, sentences, paragraphs, lines)
Paragraph Length (characters, words, sentences)
Sentence Length (characters, words)
List every word (as per a pre-defined tokenizer) and the count 

List every character used in the book

Compute min, mean, median, max and std length for words, sentences and paragraphs (this can be done after a pre-processing)

Save all that to a zip file for each processed file with the name: "original_filename"+analysis.zip 

Save size stats somewhere else to keep track and see if I can share all of it

Graph the stats here in this notebook for the general by Language

Using SpaCy for the tokenization

In [150]:
#general imports for file manipulation
import os
import sys
import zipfile  # to read the zipped gutenberg text files
import rdflib  # to read the rdf directory of the gutenberg files
from pathlib import Path  # to deal with file paths, naming and other things in a platform independent manner

In [151]:
# NLP imports
import spacy

In [152]:
# language imports
from pycountry import languages  # to deal with language naming conventions

In [153]:
# multiprocessing imports
from multiprocessing import Pool

In [154]:
# local tools imports
from utils import get_all_files_recurse, path_leaf
# gutenberg project metadata parsing
from metainfo import * # slightly modified script so it works for what I need


In [155]:
# gutenberg specific imports
import gutenberg_cleaner  #to clean headers and footers from gutenberg files, they are NOISE
# redefine the naming .. just because -> TODO take this out
gcleaner = gutenberg_cleaner.simple_cleaner

In [6]:
gcleaner?

Signature: gcleaner(book: str) -> str
Docstring:
Just removes lines that are part of the Project Gutenberg header or footer.
Doesnt go deeply in the text to remove other things like titles or footnotes or etc.
:rtype: str
:param book: str of a gutenberg's book
:return: str of the book without the lines that are part of the Project Gutenberg header and footer.
File:      ~/venv3/lib/python3.8/site-packages/gutenberg_cleaner-0.1.6-py3.8.egg/_cleaning_options/cleaner.py
Type:      function


In [81]:
# listing of all the files, from https://www.gutenberg.org/dirs/GUTINDEX.zip
GUTINDEX = "/media/nfs/Datasets/text/Gutenberg/GUTINDEX.ALL"
GUTINDEX_ZIP = "/media/nfs/Datasets/text/Gutenberg/GUTINDEX.zip"
with open(GUTINDEX, "r") as f:
    gutindex = f.read()


In [86]:
begin_idx,end_idx = gutindex.find('<==LISTINGS==>'), gutindex.find('<==End of GUTINDEX.ALL==>')


In [89]:
gutindex_db = gutindex[begin_idx:end_idx]

In [90]:
gutindex_db[:100]

'<==LISTINGS==>\n\n**** A C Following a Project Gutenberg eBook Number Indicates Copyright ****\n\n   ***'

In [91]:
gutindex_lines = gutindex_db.split('\n\n')

In [92]:
len(gutindex_lines)

55328

Is it good enough, there are missing files?? or is just that there are many duplicate files from the zip files?
Are all the files present in this listing?

Trying to get the RDF/XML database but seems to be a bitch 

In [9]:
# where the Gutenberg project file dump is
BASE_DIR = "/media/nfs/Datasets/text/Gutenberg/aleph.gutenberg.org"


In [104]:
RDF_TAR_FILE = "/media/nfs/Datasets/text/Gutenberg/rdf-files.tar.bz2"
BASE_RDF_DIR = "/media/nfs/Datasets/text/Gutenberg/rdf_db/cache/epub"

In [26]:
# get all files
#gutfiles = get_all_files_recurse(BASE_DIR)
# gutfiles = [ f for f in gutfiles if f.endswith('.zip')]
# I already had a complete list done in a txt file
with open("/media/nfs/Datasets/text/Gutenberg/zip_list.txt", "r") as f:
    gutfiles = f.readlines()

In [27]:
len(gutfiles)

91793

In [28]:
gutfiles[0].split(" ")

['[', '45K]', '', 'aleph.gutenberg.org/0/1/1-0.zip\n']

In [29]:
gutfiles = [ i.split("  ")[1].replace('\n', '') for i in gutfiles]

In [31]:
gutfiles[1000:1010]

['aleph.gutenberg.org/1/0/6/7/10673/10673.zip',
 'aleph.gutenberg.org/1/0/6/7/10674/10674-8.zip',
 'aleph.gutenberg.org/1/0/6/7/10674/10674.zip',
 'aleph.gutenberg.org/1/0/6/7/10675/10675-8.zip',
 'aleph.gutenberg.org/1/0/6/7/10675/10675.zip',
 'aleph.gutenberg.org/1/0/6/7/10676/10676-8.zip',
 'aleph.gutenberg.org/1/0/6/7/10676/10676.zip',
 'aleph.gutenberg.org/1/0/6/7/10677/10677-8.zip',
 'aleph.gutenberg.org/1/0/6/7/10677/10677.zip',
 'aleph.gutenberg.org/1/0/6/7/10678/10678-8.zip']

In [108]:
# %%time
# rdf_files = [f for f in get_all_files_recurse(BASE_RDF_DIR) if f.endswith(".rdf")]

# no need as the later script

In [99]:
testrdf = BASE_RDF_DIR+"/1800/pg1800.rdf"


In [100]:
rdfg = rdflib.Graph()
rdfg.parse(testrdf)

<Graph identifier=Nea02597699484e969ee9844a071f953e (<class 'rdflib.graph.Graph'>)>

In [102]:
for s,p,o in rdfg:
    print(' | '.join([o,p,s]))

http://purl.org/dc/terms/IMT | http://purl.org/dc/dcam/memberOf | N2ee9868a66b14bbf8d893a733da94d88
http://www.gutenberg.org/ebooks/1800 | http://purl.org/dc/terms/isFormatOf | http://www.gutenberg.org/files/1800/1800.zip
http://www.gutenberg.org/ebooks/1800.rdf | http://purl.org/dc/terms/hasFormat | http://www.gutenberg.org/ebooks/1800
3451 | http://purl.org/dc/terms/extent | http://www.gutenberg.org/cache/epub/1800/pg1800.cover.small.jpg
Copyrighted. Read the copyright notice inside this book for details. | http://purl.org/dc/terms/rights | http://www.gutenberg.org/ebooks/1800
http://www.gutenberg.org/2009/pgterms/file | http://www.w3.org/1999/02/22-rdf-syntax-ns#type | http://www.gutenberg.org/ebooks/1800.rdf
http://www.gutenberg.org/2009/pgterms/file | http://www.w3.org/1999/02/22-rdf-syntax-ns#type | http://www.gutenberg.org/cache/epub/1800/pg1800.cover.small.jpg
214089 | http://purl.org/dc/terms/extent | http://www.gutenberg.org/ebooks/1800.html.images
application/zip | http://ww

Analysis of some things where I can find the data I need, examples follow for a few elements: 

Triplets are: Object | Predicate | Subject

    text/plain; charset=us-ascii | http://www.w3.org/1999/02/22-rdf-syntax-ns#value | N0ddd72365b03404794f17691a9d033dd
    Shakspeare, William | http://www.gutenberg.org/2009/pgterms/alias | http://www.gutenberg.org/2009/agents/65
    Tragicomedy | http://www.w3.org/1999/02/22-rdf-syntax-ns#value | N48ba9daae4da4315a71f97515de1ac3d
    en | http://www.w3.org/1999/02/22-rdf-syntax-ns#value | Ne86faea53a7a46c3aa2126391e768c5d
    Married people -- Drama | http://www.w3.org/1999/02/22-rdf-syntax-ns#value | Na02a77874275488c823e1cf9c1af1e5d
    Shakespeare, William | http://www.gutenberg.org/2009/pgterms/name | http://www.gutenberg.org/2009/agents/65
    text/plain; charset=us-ascii | http://www.w3.org/1999/02/22-rdf-syntax-ns#value | Neab2b458666341749abaad5cc1b867d7
    http://www.gutenberg.org/ebooks/1800.epub.noimages | http://purl.org/dc/terms/hasFormat | http://www.gutenberg.org/ebooks/1800
    Castaways -- Drama | http://www.w3.org/1999/02/22-rdf-syntax-ns#value | N22b4e5001dbc4efb9196fc7601eb19b4
    http://www.gutenberg.org/ebooks/1800 | http://purl.org/dc/terms/isFormatOf | http://www.gutenberg.org/ebooks/1800.rdf
    The Winter's Tale | http://purl.org/dc/terms/title | http://www.gutenberg.org/ebooks/1800

There seems to be a nice [Gist here](https://gist.github.com/andreasvc/b3b4189120d84dec8857) to deal with RDF data form Gutenberg
that I've cloned [here](https://gist.github.com/leomrocha/23ac4a4b4f4d365502c9e32bee46b797)


In [103]:
# gutenberg project metadata parsing
from metainfo import *

# slightly modified script so it works for what I need


In [105]:
%%time
gut_metadata = readmetadata(RDF_TAR_FILE)

CPU times: user 1min 5s, sys: 0 ns, total: 1min 5s
Wall time: 1min 5s


In [106]:
gut_metadata[1800]

{'id': 1800,
 'author': 'Shakespeare, William',
 'title': "The Winter's Tale",
 'downloads': 8,
 'formats': {'application/epub+zip': 'http://www.gutenberg.org/ebooks/1800.epub.images',
  'application/x-mobipocket-ebook': 'http://www.gutenberg.org/ebooks/1800.kindle.images',
  'application/rdf+xml': 'http://www.gutenberg.org/ebooks/1800.rdf',
  'text/plain': 'http://www.gutenberg.org/ebooks/1800.txt.utf-8',
  'image/jpeg': 'http://www.gutenberg.org/cache/epub/1800/pg1800.cover.small.jpg',
  'text/plain; charset=us-ascii': 'http://www.gutenberg.org/files/1800/1800.txt',
  'text/html': 'http://www.gutenberg.org/ebooks/1800.html.noimages',
  'application/zip': 'http://www.gutenberg.org/files/1800/1800.zip'},
 'type': 'Text',
 'LCC': {'PR'},
 'subjects': {'Castaways -- Drama',
  'Fathers and daughters -- Drama',
  'Married people -- Drama',
  'Sicily (Italy) -- Kings and rulers -- Drama',
  'Tragicomedy'},
 'authoryearofbirth': 1564,
 'authoryearofdeath': 1616,
 'language': ['en']}

In [107]:
gut_metadata[18092]

{'id': 18092,
 'author': 'About, Edmond',
 'title': 'Germaine',
 'downloads': 95,
 'formats': {'text/plain; charset=iso-8859-1': 'http://www.gutenberg.org/files/18092/18092-8.zip',
  'application/epub+zip': 'http://www.gutenberg.org/ebooks/18092.epub.noimages',
  'text/html': 'http://www.gutenberg.org/ebooks/18092.html.noimages',
  'image/jpeg': 'http://www.gutenberg.org/cache/epub/18092/pg18092.cover.medium.jpg',
  'application/rdf+xml': 'http://www.gutenberg.org/ebooks/18092.rdf',
  'application/x-mobipocket-ebook': 'http://www.gutenberg.org/ebooks/18092.kindle.images',
  'text/plain': 'http://www.gutenberg.org/ebooks/18092.txt.utf-8'},
 'type': 'Text',
 'LCC': {'PQ'},
 'subjects': {'French fiction -- 19th century'},
 'authoryearofbirth': 1828,
 'authoryearofdeath': 1885,
 'language': ['fr']}

In [110]:
gut_metadata[20775]

{'id': 20775,
 'author': 'Arana Xajilá, Francisco Hernández',
 'title': 'The Annals of the Cakchiquels',
 'downloads': 249,
 'formats': {'application/rdf+xml': 'http://www.gutenberg.org/ebooks/20775.rdf',
  'text/plain; charset=iso-8859-1': 'http://www.gutenberg.org/files/20775/20775-8.zip',
  'text/html; charset=iso-8859-1': 'http://www.gutenberg.org/files/20775/20775-h/20775-h.htm',
  'application/x-mobipocket-ebook': 'http://www.gutenberg.org/ebooks/20775.kindle.images',
  'image/jpeg': 'http://www.gutenberg.org/cache/epub/20775/pg20775.cover.medium.jpg',
  'application/epub+zip': 'http://www.gutenberg.org/ebooks/20775.epub.noimages',
  'text/plain; charset=utf-8': 'http://www.gutenberg.org/files/20775/20775-0.txt',
  'application/zip': 'http://www.gutenberg.org/files/20775/20775-0.zip',
  'text/plain; charset=us-ascii': 'http://www.gutenberg.org/files/20775/20775.txt',
  'application/octet-stream': 'http://www.gutenberg.org/files/20775/20775-page-images.zip'},
 'type': 'Text',
 'LC

In [109]:
gut_metadata[9000]

{'id': 9000,
 'author': None,
 'title': 'Sri Vishnu Sahasranaamam',
 'downloads': 163,
 'formats': {'text/plain': 'http://www.gutenberg.org/ebooks/9000.txt.utf-8',
  'application/epub+zip': 'http://www.gutenberg.org/ebooks/9000.epub.images',
  'application/rdf+xml': 'http://www.gutenberg.org/ebooks/9000.rdf',
  'application/x-mobipocket-ebook': 'http://www.gutenberg.org/ebooks/9000.kindle.noimages',
  'text/plain; charset=us-ascii': 'http://www.gutenberg.org/files/9000/9000.txt',
  'text/html; charset=iso-8859-1': 'http://www.gutenberg.org/files/9000/9000-h/9000-h.htm',
  'image/jpeg': 'http://www.gutenberg.org/cache/epub/9000/pg9000.cover.medium.jpg'},
 'type': 'Text',
 'LCC': {'BL'},
 'subjects': {'Vishnu (Hindu deity)'},
 'authoryearofbirth': None,
 'authoryearofdeath': None,
 'language': ['sa']}

In [111]:
gut_metadata[30774]

{'id': 30774,
 'author': 'Apostol, P. N. (Pavel Natanovich)',
 'title': 'Московия в представлении иностранцев XVI-XVII в.',
 'downloads': 124,
 'formats': {'text/plain; charset=utf-8': 'http://www.gutenberg.org/files/30774/30774-0.txt',
  'application/rdf+xml': 'http://www.gutenberg.org/ebooks/30774.rdf',
  'application/epub+zip': 'http://www.gutenberg.org/ebooks/30774.epub.images',
  'application/zip': 'http://www.gutenberg.org/files/30774/30774-h.zip',
  'image/jpeg': 'http://www.gutenberg.org/cache/epub/30774/pg30774.cover.medium.jpg',
  'application/x-mobipocket-ebook': 'http://www.gutenberg.org/ebooks/30774.kindle.noimages',
  'text/html; charset=utf-8': 'http://www.gutenberg.org/files/30774/30774-h/30774-h.htm'},
 'type': 'Text',
 'LCC': {'DK'},
 'subjects': {'Russia -- Description and travel'},
 'authoryearofbirth': 1872,
 'authoryearofdeath': 1942,
 'language': ['ru']}

In [112]:
gut_metadata[24225]

{'id': 24225,
 'author': 'Aiyuezhuren',
 'title': '戲中戲',
 'downloads': 41,
 'formats': {'application/zip': 'http://www.gutenberg.org/files/24225/24225-0.zip',
  'text/plain; charset=utf-8': 'http://www.gutenberg.org/files/24225/24225-0.txt',
  'text/html': 'http://www.gutenberg.org/ebooks/24225.html.noimages',
  'application/epub+zip': 'http://www.gutenberg.org/ebooks/24225.epub.noimages',
  'image/jpeg': 'http://www.gutenberg.org/cache/epub/24225/pg24225.cover.medium.jpg',
  'application/x-mobipocket-ebook': 'http://www.gutenberg.org/ebooks/24225.kindle.images',
  'application/rdf+xml': 'http://www.gutenberg.org/ebooks/24225.rdf'},
 'type': 'Text',
 'LCC': {'PL'},
 'subjects': set(),
 'authoryearofbirth': None,
 'authoryearofdeath': None,
 'language': ['zh']}

In [113]:
gut_metadata[1982]

{'id': 1982,
 'author': 'Akutagawa, Ryunosuke',
 'title': '羅生門',
 'downloads': 578,
 'formats': {'application/epub+zip': 'http://www.gutenberg.org/ebooks/1982.epub.noimages',
  'text/plain; charset=utf-8': 'http://www.gutenberg.org/files/1982/1982-0.txt',
  'text/html': 'http://www.gutenberg.org/ebooks/1982.html.images',
  'image/jpeg': 'http://www.gutenberg.org/cache/epub/1982/pg1982.cover.small.jpg',
  'application/rdf+xml': 'http://www.gutenberg.org/ebooks/1982.rdf',
  'application/x-mobipocket-ebook': 'http://www.gutenberg.org/ebooks/1982.kindle.images',
  'application/zip': 'http://www.gutenberg.org/files/1982/1982-0.zip'},
 'type': 'Text',
 'LCC': {'PL'},
 'subjects': {'Japan -- Social life and customs -- Fiction', 'Short stories'},
 'authoryearofbirth': 1892,
 'authoryearofdeath': 1927,
 'language': ['ja']}

In [114]:
gut_metadata[10257]

{'id': 10257,
 'author': "I. J. Hochman's Yiddisher Orchester",
 'title': 'Mazel Tov',
 'downloads': 47,
 'formats': {'audio/mpeg': 'http://www.gutenberg.org/files/10257/10257-m/10257-m-001.mp3',
  'text/plain; charset=us-ascii': 'http://www.gutenberg.org/files/10257/10257-m/10257-m-readme.txt',
  'application/rdf+xml': 'http://www.gutenberg.org/ebooks/10257.rdf'},
 'type': 'Sound',
 'LCC': {'M'},
 'subjects': {'Jews -- Music', 'Klezmer music'},
 'authoryearofbirth': None,
 'authoryearofdeath': None,
 'language': ['yi']}

In [115]:
gut_metadata[50430]

{'id': 50430,
 'author': 'Acosta, José de',
 'title': 'Historia natural y moral de las Indias (vol 2 of 2)',
 'downloads': 160,
 'formats': {'image/jpeg': 'http://www.gutenberg.org/cache/epub/50430/pg50430.cover.medium.jpg',
  'application/zip': 'http://www.gutenberg.org/files/50430/50430-0.zip',
  'application/x-mobipocket-ebook': 'http://www.gutenberg.org/ebooks/50430.kindle.noimages',
  'application/epub+zip': 'http://www.gutenberg.org/ebooks/50430.epub.images',
  'text/html; charset=iso-8859-1': 'http://www.gutenberg.org/files/50430/50430-h.zip',
  'text/plain; charset=utf-8': 'http://www.gutenberg.org/files/50430/50430-0.txt',
  'application/rdf+xml': 'http://www.gutenberg.org/ebooks/50430.rdf'},
 'type': 'Text',
 'LCC': {'E011'},
 'subjects': {'Acosta, José de, 1540-1600 -- Travel -- America',
  'America -- Description and travel',
  'America -- Early accounts to 1600',
  'Indians of Mexico -- Early works to 1800',
  'Indians of South America -- Early works to 1800',
  'Natural h

It seems that the library as is with the given path works well and extracts more information than needed. This is good for making the metadata extraction



### Pipeline description:

1. get next file
2. get file ID and base name of the file
3. unzip it in memory and get ENCODING (if available) from the file (maybe will have to reload the file ... for the moment don't care and I'll treat everything as utf-8 compatible as the encodings I found sampling some languages including tagalog, chinese, japanese seem to be utf-8 compatible)
4. Get author (name, alias, dates ... ), title and language, publishing date and editor (for the data available)
5. clean gutenberg text
6. count total length of the text
7. Separate paragraphs I'll do it by at least a `\n\n` sequence (might not work for every language .. (I do care but I cant, so somebody that knows the language should correct those) 
8. count the number of paragraphs
9. separate sentences (again, language depending, I'll do it ~by `.` characters and~ with spacy models ...
10. count the number of sentences per paragraph
11. separate words -> maybe here would need lemmatization to see some things correctly, but will have issues in non supported languages.
12. count number of words per sentence
13. sum number of words per paragraph
13. separate chars
14. count chars per word, sentence and paragraph
15. aggregate all words and count the number of occurrences
16. aggregate all the characters and count the number of occurrences
17. aggregate and sort results then save zip file with it
    

#### Notes:

There are many issues with the current data analysis, but for the global analysis the numbers should be OK enough.
These issues are caused by noise in the data, for example captions and references to images that are not in the current text (and can not be taken into account here)

There are many languages that are not supported by SpaCy, and so the tokenization might not be extremely good.

Different writing styles are present.

The current analysis' goal is to understand some basic statistics on text from different Languages and Authors, starting to give a more thourough insight on some variations that come into play when deciding implementation specificities for NLP pipelines. 

The contributions of the current study are:
- An Open Source library to analyze the entire Gutenberg project statistics.
- A first introspection on some 
- A (text) database output containing many of the elements needed for the current analysis, which in future works can be reused as a base for improved insights.

As an extra element the code separation is done in a way to allow for some code reusability in other text datasets.

SpaCy has different models, but there are not for every language, so for the moment I'll try a specific model for the available ones, but the general model for the ones that are not available.

For this is that I need to first install all available models from SpaCy, if a medium (md) model is available, this one is preferred instead of the small one (sm), the large ones (lg) are going to be ignored as they are too big for the current processing needs (the needs are quite basic in the current study)

In [161]:
spacy_models_2 = {
    'de': 'de_core_news_md', # German
    'el': 'el_core_news_sm', # Greek
    'en': 'en_core_web_md', # English
    'es': 'es_core_news_md', # Spanish
    'fr': 'fr_core_news_md', # French
    'it': 'it_core_news_sm', # Italian
    'lt': 'lt_core_news_sm', # Lithuanian
    'nb': 'nb_core_news_sm', # Norwegian Bokmål
    'nl': 'nl_core_news_sm', # Dutch
    'pt': 'pt_core_news_sm', # Portuguese
    'xx': 'xx_ent_wiki_sm', # Multi-Lang
    }
        
spacy_models = {
    'german': 'de_core_news_md', # German
    'greek': 'el_core_news_sm', # Greek
    'english': 'en_core_web_md', # English
    'spanish': 'es_core_news_md', #Sspanish
    'french': 'fr_core_news_md', # French
    'italian': 'it_core_news_sm', # Italian
    'lithuanian': 'lt_core_news_sm', # Lithuanian
    'norwegian': 'nb_core_news_sm', # Norwegian Bokmål
    'dutch': 'nl_core_news_sm', # Dutch
    'portuguese': 'pt_core_news_sm', # Portuguese
    'multi-lang': 'xx_ent_wiki_sm', # Multi-Lang
    }

In [168]:
# %%time
# for model in spacy_models.values():
#     # install the models:
#     cmd = "python -m spacy download {}".format(model)
#     print(cmd)
#     os.system(cmd)

In [44]:
spacy.info()


============================== Info about spaCy ==============================

spaCy version    2.2.4                         
Location         /home/leo/venv3/lib/python3.8/site-packages/spacy
Platform         Linux-5.4.0-29-generic-x86_64-with-glibc2.29
Python version   3.8.2                         
Models                                         



{'spaCy version': '2.2.4',
 'Location': '/home/leo/venv3/lib/python3.8/site-packages/spacy',
 'Platform': 'Linux-5.4.0-29-generic-x86_64-with-glibc2.29',
 'Python version': '3.8.2',
 'Models': ''}

In [58]:
ftest = '/media/nfs/Datasets/text/Gutenberg/aleph.gutenberg.org/1/8/0/1800/1800.zip'
ftxt = path_leaf(ftest).replace(".zip", ".txt")
f = zipfile.ZipFile(ftest)


In [125]:
ntpath.split(ftest)

('/media/nfs/Datasets/text/Gutenberg/aleph.gutenberg.org/1/8/0/1800',
 '1800.zip')

In [130]:
ntpath.dirname(ftest)

'/media/nfs/Datasets/text/Gutenberg/aleph.gutenberg.org/1/8/0/1800'

In [132]:
os.path.split(ftest), os.path.split('/media/nfs/Datasets/text/Gutenberg/aleph.gutenberg.org/1/8/0/1800')

(('/media/nfs/Datasets/text/Gutenberg/aleph.gutenberg.org/1/8/0/1800',
  '1800.zip'),
 ('/media/nfs/Datasets/text/Gutenberg/aleph.gutenberg.org/1/8/0', '1800'))

In [134]:
os.path.basename(ftest)

'1800.zip'

In [138]:
pth = Path(ftest)

In [143]:
pth.parent.name

'1800'

In [144]:
def get_file_id(fname):
    """Returns the Gutenberg File ID"""
    pth = Path(fname)
    # as per file structure the filename has some variations but the parent folder is always the ID
    return pth.parent.name

In [148]:
a = []
isinstance(a,list)

True

In [149]:
# def get_meta(gut_id, rfd_meta):
#     """ Loads the most adapted spacy nlp resource from the available ones
#     :param gut_id: Numeric gutenberg id of the file
#     :param rfd_meta:  rfd metainfo database
#     :return:  dict_metainfo
#     """
#     metainfo = rfd_meta[gut_id]
    
def get_nlp_resource(metainfo):
    
    lang = 'xx'
    try:
        lng = metainfo['language']
        if isinstance(lng, list) or isinstance(lng, tuple) and len(lng > 0):
            lng = lng[0]
        elif isinstance(lng, str):
            pass  # nothing to do here, move along
        else:
            pass # FUUUUUUU something wrong, but the default value will be multilang anyways
    except:
        # just to avoid issues if there is no language tag, in that case go back to default
        pass
    # loading with shortcut ... maybe will need to use the spacy models dict that I've created earlier, we'll see
    nlp = spacy.load(lang)
    return 

In [156]:
def process_file(fname, rfd_meta):
    # Gutenberg file id 
    f_id = get_file_id(fname)
    # meta information extracted from the Gutenber RFD database -> warning is around 1GB the DB
    metainfo = rfd_meta[gut_id]
    # TODO extract format from metadata (if exists)
    encoding = 'utf-8'  # asume all is compatible with utf-8, for the moment haven't found one that is not
    # spacy 
    nlp = get_nlp_resource(metainfo)
    # load and clean the file, asume zip as compressing format
    pth = Path(fname)  # 
    ftxt = pth.name.replace(".zip", ".txt")
    f = zipfile.ZipFile(fname)
    txt = f.read(ftxt).decode(encoding)
    txt = gutenberg_cleaner.simple_cleaner(txt)
    # Start analysis
    doc = nlp(txt)  #SpaCy tokenization
    
    sentences = list(doc.sents)
    
    stats = {}
    stats['total_char_len'] = len(txt)
    
    # count number of paragraphs .. might not work on many languages
    paragraphs =[l.strip() for l in ctxt.split('\n\n') if len(l.strip()) > 0]
    plen = len(paragraphs)
    # 
    
    #stats['paragraph'] = { }
    

In [64]:
txt = f.read(ftxt).decode('utf-8')

In [65]:
len(txt)

172007

In [66]:
type(txt)

str

In [67]:
ctxt = gcleaner(txt)

In [71]:
len(ctxt), type(ctxt)

(155798, str)

In [76]:
END_OF_HEADER = "START OF THIS PROJECT GUTENBERG EBOOK"

In [77]:
txt.find(END_OF_HEADER)

-1

In [73]:
txt

'\r\n*******************************************************************\r\nTHIS EBOOK WAS ONE OF PROJECT GUTENBERG\'S EARLY FILES PRODUCED AT A\r\nTIME WHEN PROOFING METHODS AND TOOLS WERE NOT WELL DEVELOPED. THERE\r\nIS AN IMPROVED EDITION OF THIS TITLE WHICH MAY BE VIEWED AS EBOOK\r\n(#1539) at https://www.gutenberg.org/ebooks/1539\r\n*******************************************************************\r\n\r\n\r\nThis Etext file is presented by Project Gutenberg, in\r\ncooperation with World Library, Inc., from their Library of the\r\nFuture and Shakespeare CDROMS.  Project Gutenberg often releases\r\nEtexts that are NOT placed in the Public Domain!!\r\n\r\n*This Etext has certain copyright implications you should read!*\r\n\r\n<<THIS ELECTRONIC VERSION OF THE COMPLETE WORKS OF WILLIAM\r\nSHAKESPEARE IS COPYRIGHT 1990-1993 BY WORLD LIBRARY, INC., AND IS\r\nPROVIDED BY PROJECT GUTENBERG WITH PERMISSION.  ELECTRONIC AND\r\nMACHINE READABLE COPIES MAY BE DISTRIBUTED SO LONG AS SUCH COPI

In [72]:
ctxt

'\n\n\n\n\n1611\n\nTHE WINTER\'S TALE\n\nby William Shakespeare\n\n\n\nDramatis Personae\n\n  LEONTES, King of Sicilia\n  MAMILLIUS, his son, the young Prince of Sicilia\n  CAMILLO,    lord of Sicilia\n  ANTIGONUS,    "   "     "\n  CLEOMENES,    "   "     "\n  DION,         "   "     "\n  POLIXENES, King of Bohemia\n  FLORIZEL, his son, Prince of Bohemia\n  ARCHIDAMUS, a lord of Bohemia\n  OLD SHEPHERD, reputed father of Perdita\n  CLOWN, his son\n  AUTOLYCUS, a rogue\n  A MARINER\n  A GAOLER\n  TIME, as Chorus\n\n  HERMIONE, Queen to Leontes\n  PERDITA, daughter to Leontes and Hermione\n  PAULINA, wife to Antigonus\n  EMILIA, a lady attending on the Queen\n  MOPSA,   shepherdess\n  DORCAS,        "\n\n  Other Lords, Gentlemen, Ladies, Officers, Servants, Shepherds,\n    Shepherdesses\n\n                              SCENE:\n                       Sicilia and Bohemia\n\n\n\n\n\n\n\nACT I. SCENE I.\nSicilia. The palace of LEONTES\n\nEnter CAMILLO and ARCHIDAMUS\n\n  ARCHIDAMUS. If you 

In [116]:
paragraphs =[l.strip() for l in ctxt.split('\n\n') if len(l.strip()) > 0]

In [117]:
len(paragraphs)

126

In [121]:
paragraphs[:15]

['1611',
 "THE WINTER'S TALE",
 'by William Shakespeare',
 'Dramatis Personae',
 'LEONTES, King of Sicilia\n  MAMILLIUS, his son, the young Prince of Sicilia\n  CAMILLO,    lord of Sicilia\n  ANTIGONUS,    "   "     "\n  CLEOMENES,    "   "     "\n  DION,         "   "     "\n  POLIXENES, King of Bohemia\n  FLORIZEL, his son, Prince of Bohemia\n  ARCHIDAMUS, a lord of Bohemia\n  OLD SHEPHERD, reputed father of Perdita\n  CLOWN, his son\n  AUTOLYCUS, a rogue\n  A MARINER\n  A GAOLER\n  TIME, as Chorus',
 'HERMIONE, Queen to Leontes\n  PERDITA, daughter to Leontes and Hermione\n  PAULINA, wife to Antigonus\n  EMILIA, a lady attending on the Queen\n  MOPSA,   shepherdess\n  DORCAS,        "',
 'Other Lords, Gentlemen, Ladies, Officers, Servants, Shepherds,\n    Shepherdesses',
 'SCENE:\n                       Sicilia and Bohemia',
 'ACT I. SCENE I.\nSicilia. The palace of LEONTES',
 'Enter CAMILLO and ARCHIDAMUS',
 "ARCHIDAMUS. If you shall chance, Camillo, to visit Bohemia, on the\n    l

In [164]:
nlp = spacy.load(spacy_models_2['en'])

OSError: [E050] Can't find model 'en_core_web_md'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.

In [166]:
# Well there is an issue to fix with SpaCy now
# spacy.load('en_core_web_md')
import en_core_web_md

In [167]:
nlp = en_core_web_md.load()

In [169]:
%%time
doc = nlp(ctxt)

CPU times: user 2.82 s, sys: 409 ms, total: 3.23 s
Wall time: 3.23 s


In [170]:
sentences = list(doc.sents)  

In [171]:
len(sentences)

3726

In [177]:
type(sentences[100]), len(sentences[100]), sentences[100]

(spacy.tokens.span.Span,
 15,
 Say this to him,
     He's beat from his best ward.
   )

In [183]:
vocab_list = list(doc.vocab.strings)

In [185]:
len(vocab_list), len(set(vocab_list)),

(1476693, 1476693)